<a href="https://colab.research.google.com/github/dngoins/Time-LLM/blob/main/TimeLLM_ETTh1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# requires libuvc
# pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126


 # use this instead
 #pip3 install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu118

# first install pytorch with command above
# this worked with python version 3.12.5

# so far only working on a Linux box
# use pip3 install -r requirements.txt to install all dependencies
!pip3 install torch>=2.3.0 torchvision>=0.18.0 torchaudio>=2.3.0

!pip3 install accelerate==0.28.0 einops>=0.7.0 matplotlib>=3.7.0 numpy
!pip3 install pandas scikit_learn scipy tqdm
!pip3 install peft>=0.4.0 transformers deepspeed
!pip3 install sentencepiece>=0.2.0 setuptools
!pip3 install mpi4py
!pip install azure-storage-blob azure-identity


In [2]:
!pip install --upgrade accelerate

  Using cached accelerate-1.5.1-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.5.1-py3-none-any.whl (345 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0


In [3]:
from google.colab import userdata

# Set your Azure Storage connection string as a secret
# This needs to be done in Colab's UI under "Secrets" tab
#connection_string = """DefaultEndpointsProtocol=https;AccountName=;AccountKey=;EndpointSuffix=core.windows.net"""

#userdata.set('AZURE_STORAGE_CONNECTION_STRING', connection_string)

In [4]:

# Array math
import numpy as np

# Iteration tracking
from tqdm import tqdm

# Type hinting
from typing import Tuple

# Os traversal
import os

import numpy as np
import random

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from google.colab import drive
from google.colab import files
import io
from IPython.display import display, Image
from datetime import datetime  # Import datetime module
import time
import pandas as pd

import matplotlib.animation as animation
from matplotlib.patches import FancyArrowPatch

from sklearn.preprocessing import MinMaxScaler

import argparse
import torch
from accelerate import Accelerator, DeepSpeedPlugin
from accelerate import DistributedDataParallelKwargs
from torch import nn, optim
from torch.optim import lr_scheduler
from tqdm import tqdm

import json


In [5]:
# Mounting G-Drive for saving files.
drive.mount('/content/gdrive')
timeLLM_path = '/content/gdrive/MyDrive/TimeLLM'

%cd {timeLLM_path}


Mounted at /content/gdrive
/content/gdrive/MyDrive/TimeLLM


In [6]:
from math import sqrt


from accelerate import Accelerator, DeepSpeedPlugin
from accelerate import DistributedDataParallelKwargs
from torch import nn, optim
from torch.optim import lr_scheduler
from tqdm import tqdm

import json
import torch
import torch.nn as nn

from transformers import LlamaConfig, LlamaModel, LlamaTokenizer, GPT2Config, GPT2Model, GPT2Tokenizer, BertConfig, BertModel, BertTokenizer

from layers.Embed import PatchEmbedding
import transformers

from layers.StandardNorm import Normalize

from models import Autoformer, DLinear
from transformers import LlamaTokenizer, LlamaModel

from data_provider.data_factory import data_provider
import time
import random
import numpy as np
import os

os.environ['CURL_CA_BUNDLE'] = ''
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

from utils.tools import del_files, EarlyStopping, adjust_learning_rate, vali, load_content

transformers.logging.set_verbosity_error()


class FlattenHead(nn.Module):
    def __init__(self, n_vars, nf, target_window, head_dropout=0):
        super().__init__()
        self.n_vars = n_vars
        self.flatten = nn.Flatten(start_dim=-2)
        self.linear = nn.Linear(nf, target_window)
        self.dropout = nn.Dropout(head_dropout)

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        x = self.dropout(x)
        return x


class TimeLLM(nn.Module):

    def __init__(self, configs, patch_len=16, stride=8):
        super(TimeLLM, self).__init__()

        print('TimeLLM init')
        self.task_name = configs.task_name
        self.pred_len = configs.pred_len
        self.seq_len = configs.seq_len
        self.d_ff = configs.d_ff
        self.top_k = 5
        self.d_llm = configs.llm_dim
        self.patch_len = configs.patch_len
        self.stride = configs.stride

        if configs.llm_model == 'LLAMA':
            # self.llama_config = LlamaConfig.from_pretrained('/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/')
            self.llama_config = LlamaConfig.from_pretrained('huggyllama/llama-7b')
            self.llama_config.num_hidden_layers = configs.llm_layers
            self.llama_config.output_attentions = True
            self.llama_config.output_hidden_states = True

            print('TimeLLM- LLAMA')
            try:
                # self.llm_model = LlamaModel.from_pretrained(
                #     # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/",
                #     'huggyllama/llama-7b',
                #     trust_remote_code=True,
                #     local_files_only=True,
                #     config=self.llama_config,
                #     use_safetensors=False
                #     # load_in_4bit=True
                # )
                self.llm_model = LlamaModel.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    # local_files_only=True, # This line is causing the issue
                    config=self.llama_config,
                    use_safetensors=False
                    # load_in_4bit=True
                )
                print('TimeLLM-model from pretrained')
            except EnvironmentError:  # downloads model from HF is not already done
                print("Local model files not found. Attempting to download...")
                self.llm_model = LlamaModel.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    # local_files_only=True, # This line is causing the issue
                    config=self.llama_config,
                    use_safetensors=False
                    # load_in_4bit=True
                )
            try:
                self.tokenizer = LlamaTokenizer.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/tokenizer.model",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    local_files_only=True
                )

                print('TimeLLM-tokenizer from pretrained')
            except EnvironmentError:  # downloads the tokenizer from HF if not already done
                print("Local tokenizer files not found. Atempting to download them..")
                self.tokenizer = LlamaTokenizer.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/tokenizer.model",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    local_files_only=False
                )
        elif configs.llm_model == 'GPT2':
            self.gpt2_config = GPT2Config.from_pretrained('openai-community/gpt2')

            self.gpt2_config.num_hidden_layers = configs.llm_layers
            self.gpt2_config.output_attentions = True
            self.gpt2_config.output_hidden_states = True
            print('TimeLLM- GPT2')
            try:
                self.llm_model = GPT2Model.from_pretrained(
                    'openai-community/gpt2',
                    config=self.gpt2_config,
                    trust_remote_code=True,
                    # local_files_only=True, # This line is causing the issue
                    use_safetensors=False
                    # load_in_4bit=True
                )
            except EnvironmentError:  # downloads model from HF is not already done
                print("Local model files not found. Attempting to download...")
                self.llm_model = GPT2Model.from_pretrained(
                    'openai-community/gpt2',
                    config=self.gpt2_config,
                    trust_remote_code=True,
                    # local_files_only=True, # This line is causing the issue
                    use_safetensors=False
                    # load_in_4bit=True
                )

            try:
                self.tokenizer = GPT2Tokenizer.from_pretrained(
                    'openai-community/gpt2',
                    trust_remote_code=True,
                    local_files_only=True
                )
            except EnvironmentError:  # downloads the tokenizer from HF if not already done
                print("Local tokenizer files not found. Atempting to download them..")
                self.tokenizer = GPT2Tokenizer.from_pretrained(
                    'openai-community/gpt2',
                    trust_remote_code=True,
                    local_files_only=False
                )
        elif configs.llm_model == 'BERT':
            self.bert_config = BertConfig.from_pretrained('google-bert/bert-base-uncased')

            self.bert_config.num_hidden_layers = configs.llm_layers
            self.bert_config.output_attentions = True
            self.bert_config.output_hidden_states = True
            print('TimeLLM- BERT')
            try:
                self.llm_model = BertModel.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    #local_files_only=True,
                    config=self.bert_config,
                    use_safetensors=False
                    # load_in_4bit=True
                )
            except EnvironmentError:  # downloads model from HF is not already done
                print("Local model files not found. Attempting to download...")
                self.llm_model = BertModel.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=False,
                    config=self.bert_config,
                    use_safetensors=False
                    # load_in_4bit=True
                )

            try:
                self.tokenizer = BertTokenizer.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=True
                )
            except EnvironmentError:  # downloads the tokenizer from HF if not already done
                print("Local tokenizer files not found. Atempting to download them..")
                self.tokenizer = BertTokenizer.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=False
                )
        else:
            raise Exception('LLM model is not defined')

        print('TimeLLM- Tokenizer')
        if self.tokenizer.eos_token:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        else:
            pad_token = '[PAD]'
            self.tokenizer.add_special_tokens({'pad_token': pad_token})
            self.tokenizer.pad_token = pad_token

        for param in self.llm_model.parameters():
            param.requires_grad = False

        if configs.prompt_domain:
            self.description = configs.content
        else:
            self.description = 'The Electricity Transformer Temperature (ETT) is a crucial indicator in the electric power long-term deployment.'

        print(f'TimeLLM-description:\n***************\n{self.description}\n***************\n')
        self.dropout = nn.Dropout(configs.dropout)

        self.patch_embedding = PatchEmbedding(
            configs.d_model, self.patch_len, self.stride, configs.dropout)

        self.word_embeddings = self.llm_model.get_input_embeddings().weight
        self.vocab_size = self.word_embeddings.shape[0]
        self.num_tokens = 1000
        self.mapping_layer = nn.Linear(self.vocab_size, self.num_tokens)

        print('Reprogramming Layer')
        self.reprogramming_layer = ReprogrammingLayer(configs.d_model, configs.n_heads, self.d_ff, self.d_llm)

        self.patch_nums = int((configs.seq_len - self.patch_len) / self.stride + 2)
        self.head_nf = self.d_ff * self.patch_nums

        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            self.output_projection = FlattenHead(configs.enc_in, self.head_nf, self.pred_len,
                                                 head_dropout=configs.dropout)
            print(f'TimeLLM-output_projection:\n***************\n{self.output_projection}\n***************\n')
        else:
            raise NotImplementedError

        self.normalize_layers = Normalize(configs.enc_in, affine=False)

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, mask=None):
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            dec_out = self.forecast(x_enc, x_mark_enc, x_dec, x_mark_dec)
            return dec_out[:, -self.pred_len:, :]
        return None

    def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):

        x_enc = self.normalize_layers(x_enc, 'norm')

        B, T, N = x_enc.size()
        x_enc = x_enc.permute(0, 2, 1).contiguous().reshape(B * N, T, 1)

        min_values = torch.min(x_enc, dim=1)[0]
        max_values = torch.max(x_enc, dim=1)[0]
        medians = torch.median(x_enc, dim=1).values
        lags = self.calcute_lags(x_enc)
        trends = x_enc.diff(dim=1).sum(dim=1)

        prompt = []
        for b in range(x_enc.shape[0]):
            min_values_str = str(min_values[b].tolist()[0])
            max_values_str = str(max_values[b].tolist()[0])
            median_values_str = str(medians[b].tolist()[0])
            lags_values_str = str(lags[b].tolist())
            prompt_ = (
                f"<|start_prompt|>Dataset description: {self.description}"
                f"Task description: forecast the next {str(self.pred_len)} steps given the previous {str(self.seq_len)} steps information; "
                "Input statistics: "
                f"min value {min_values_str}, "
                f"max value {max_values_str}, "
                f"median value {median_values_str}, "
                f"the trend of input is {'upward' if trends[b] > 0 else 'downward'}, "
                f"top 5 lags are : {lags_values_str}<|<end_prompt>|>"
            )

            prompt.append(prompt_)

        x_enc = x_enc.reshape(B, N, T).permute(0, 2, 1).contiguous()
        #print(f'*********\nPrompt:\n{prompt}\n**************\n')

        prompt = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).input_ids

        #print(f'*********\nPrompt-Tokenized:\n{prompt}\n**************\n')
        prompt_embeddings = self.llm_model.get_input_embeddings()(prompt.to(x_enc.device))  # (batch, prompt_token, dim)

        source_embeddings = self.mapping_layer(self.word_embeddings.permute(1, 0)).permute(1, 0)

        x_enc = x_enc.permute(0, 2, 1).contiguous()
        enc_out, n_vars = self.patch_embedding(x_enc.to(torch.bfloat16))
        enc_out = self.reprogramming_layer(enc_out, source_embeddings, source_embeddings)
        llama_enc_out = torch.cat([prompt_embeddings, enc_out], dim=1)
        dec_out = self.llm_model(inputs_embeds=llama_enc_out).last_hidden_state
        dec_out = dec_out[:, :, :self.d_ff]

        dec_out = torch.reshape(
            dec_out, (-1, n_vars, dec_out.shape[-2], dec_out.shape[-1]))
        dec_out = dec_out.permute(0, 1, 3, 2).contiguous()

        dec_out = self.output_projection(dec_out[:, :, :, -self.patch_nums:])
        dec_out = dec_out.permute(0, 2, 1).contiguous()

        dec_out = self.normalize_layers(dec_out, 'denorm')

        return dec_out

    def calcute_lags(self, x_enc):
        q_fft = torch.fft.rfft(x_enc.permute(0, 2, 1).contiguous(), dim=-1)
        k_fft = torch.fft.rfft(x_enc.permute(0, 2, 1).contiguous(), dim=-1)
        res = q_fft * torch.conj(k_fft)
        corr = torch.fft.irfft(res, dim=-1)
        mean_value = torch.mean(corr, dim=1)
        _, lags = torch.topk(mean_value, self.top_k, dim=-1)
        return lags


class ReprogrammingLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_keys=None, d_llm=None, attention_dropout=0.1):
        super(ReprogrammingLayer, self).__init__()

        d_keys = d_keys or (d_model // n_heads)

        self.query_projection = nn.Linear(d_model, d_keys * n_heads)
        self.key_projection = nn.Linear(d_llm, d_keys * n_heads)
        self.value_projection = nn.Linear(d_llm, d_keys * n_heads)
        self.out_projection = nn.Linear(d_keys * n_heads, d_llm)
        self.n_heads = n_heads
        self.dropout = nn.Dropout(attention_dropout)

    def forward(self, target_embedding, source_embedding, value_embedding):
        B, L, _ = target_embedding.shape
        S, _ = source_embedding.shape
        H = self.n_heads

        target_embedding = self.query_projection(target_embedding).view(B, L, H, -1)
        source_embedding = self.key_projection(source_embedding).view(S, H, -1)
        value_embedding = self.value_projection(value_embedding).view(S, H, -1)

        out = self.reprogramming(target_embedding, source_embedding, value_embedding)

        out = out.reshape(B, L, -1)

        return self.out_projection(out)

    def reprogramming(self, target_embedding, source_embedding, value_embedding):
        B, L, H, E = target_embedding.shape

        scale = 1. / sqrt(E)

        scores = torch.einsum("blhe,she->bhls", target_embedding, source_embedding)

        A = self.dropout(torch.softmax(scale * scores, dim=-1))
        reprogramming_embedding = torch.einsum("bhls,she->blhe", A, value_embedding)

        return reprogramming_embedding


In [7]:
def train(args):
  for ii in range(args.itr):
      # setting record of experiments
      setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_{}_{}'.format(
          args.task_name,
          args.model_id,
          args.model,
          args.data,
          args.features,
          args.seq_len,
          args.label_len,
          args.pred_len,
          args.d_model,
          args.n_heads,
          args.e_layers,
          args.d_layers,
          args.d_ff,
          args.factor,
          args.embed,
          args.des, ii)


      train_data, train_loader = data_provider(args, 'train')
      vali_data, vali_loader = data_provider(args, 'val')
      test_data, test_loader = data_provider(args, 'test')

      if 'Autoformer' == args.model:
          model = Autoformer.Model(args).float()
      elif 'DLinear' == args.model:
          model = DLinear.Model(args).float()
      else:
          model = TimeLLM(args).float()

      path = os.path.join(args.checkpoints,
                          setting + '-' + args.model_comment)  # unique checkpoint saving path
      args.content = load_content(args)
      print(f'***** Content *****\n{args.content}\n*****\n')

      if not os.path.exists(path) and accelerator.is_local_main_process:
          os.makedirs(path)

      time_now = time.time()

      train_steps = len(train_loader)
      early_stopping = EarlyStopping(accelerator=accelerator, patience=args.patience)
      accelerator.print(f'Early stopping patience: {args.patience}')

      trained_parameters = []
      for p in model.parameters():
          if p.requires_grad is True:
              trained_parameters.append(p)

      print(f'Words: {model.tokenizer.vocab_size}')
      print(f'Tokens: {model.num_tokens}')
      print(f'Trained parameters: {len(trained_parameters)}')
      print(f'Total parameters: {sum(p.numel() for p in model.parameters())}')
      print(f'Total parameters (trainable): {sum(p.numel() for p in trained_parameters)}')
      print(f'Total parameters (non-trainable): {sum(p.numel() for p in model.parameters()) - sum(p.numel() for p in trained_parameters)}')

      model_optim = optim.Adam(trained_parameters, lr=args.learning_rate)

      if args.lradj == 'COS':
          scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=20, eta_min=1e-8)
      else:
          scheduler = lr_scheduler.OneCycleLR(optimizer=model_optim,
                                              steps_per_epoch=train_steps,
                                              pct_start=args.pct_start,
                                              epochs=args.train_epochs,
                                              max_lr=args.learning_rate)

      criterion = nn.MSELoss()
      mae_metric = nn.L1Loss()

      train_loader, vali_loader, test_loader, model, model_optim, scheduler = accelerator.prepare(
          train_loader, vali_loader, test_loader, model, model_optim, scheduler)

      if args.use_amp:
          scaler = torch.cuda.amp.GradScaler()

      for epoch in range(args.train_epochs):
          iter_count = 0
          train_loss = []

          model.train()
          epoch_time = time.time()
          for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in tqdm(enumerate(train_loader)):
              iter_count += 1
              model_optim.zero_grad()

              batch_x = batch_x.float().to(accelerator.device)
              batch_y = batch_y.float().to(accelerator.device)
              batch_x_mark = batch_x_mark.float().to(accelerator.device)
              batch_y_mark = batch_y_mark.float().to(accelerator.device)

              # decoder input
              dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float().to(
                  accelerator.device)
              dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float().to(
                  accelerator.device)

              # encoder - decoder
              if args.use_amp:
                  with torch.cuda.amp.autocast():
                      if args.output_attention:
                          outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                      else:
                          outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

                      f_dim = -1 if args.features == 'MS' else 0
                      outputs = outputs[:, -args.pred_len:, f_dim:]
                      batch_y = batch_y[:, -args.pred_len:, f_dim:].to(accelerator.device)
                      loss = criterion(outputs, batch_y)
                      train_loss.append(loss.item())
              else:
                  if args.output_attention:
                      outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                  else:
                      outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

                  f_dim = -1 if args.features == 'MS' else 0
                  outputs = outputs[:, -args.pred_len:, f_dim:]
                  batch_y = batch_y[:, -args.pred_len:, f_dim:]
                  loss = criterion(outputs, batch_y)
                  train_loss.append(loss.item())

              if (i + 1) % 100 == 0:
                  accelerator.print(
                      "\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
                  speed = (time.time() - time_now) / iter_count
                  left_time = speed * ((args.train_epochs - epoch) * train_steps - i)
                  accelerator.print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                  iter_count = 0
                  time_now = time.time()

              if args.use_amp:
                  scaler.scale(loss).backward()
                  scaler.step(model_optim)
                  scaler.update()
              else:
                  accelerator.backward(loss)
                  model_optim.step()

              if args.lradj == 'TST':
                  adjust_learning_rate(accelerator, model_optim, scheduler, epoch + 1, args, printout=False)
                  scheduler.step()

          accelerator.print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
          train_loss = np.average(train_loss)
          vali_loss, vali_mae_loss = vali(args, accelerator, model, vali_data, vali_loader, criterion, mae_metric)
          test_loss, test_mae_loss = vali(args, accelerator, model, test_data, test_loader, criterion, mae_metric)
          accelerator.print(
              "Epoch: {0} | Train Loss: {1:.7f} Vali Loss: {2:.7f} Test Loss: {3:.7f} MAE Loss: {4:.7f}".format(
                  epoch + 1, train_loss, vali_loss, test_loss, test_mae_loss))

          drive.mount('/content/gdrive')
          timeLLM_path = '/content/gdrive/MyDrive/TimeLLM'
          %cd {timeLLM_path}

          early_stopping(vali_loss, model, path)
          if early_stopping.early_stop:
              accelerator.print("Early stopping")
              break

          if args.lradj != 'TST':
              if args.lradj == 'COS':
                  scheduler.step()
                  accelerator.print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
              else:
                  if epoch == 0:
                      args.learning_rate = model_optim.param_groups[0]['lr']
                      accelerator.print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
                  adjust_learning_rate(accelerator, model_optim, scheduler, epoch + 1, args, printout=True)

          else:
              accelerator.print('Updating learning rate to {}'.format(scheduler.get_last_lr()[0]))

          # Mounting G-Drive for saving files.

  print(f'Words: {model.tokenizer.vocab_size}')
  print(f'Tokens: {model.num_tokens}')
  print(f'Trained parameters: {len(trained_parameters)}')
  print(f'Total parameters: {sum(p.numel() for p in model.parameters())}')
  print(f'Total parameters (trainable): {sum(p.numel() for p in trained_parameters)}')
  print(f'Total parameters (non-trainable): {sum(p.numel() for p in model.parameters()) - sum(p.numel() for p in trained_parameters)}')



In [8]:

parser = argparse.ArgumentParser(description='Time-LLM')

fix_seed = 2025
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

# basic config
parser.add_argument('-f', type=str, default='')
parser.add_argument('--task_name', type=str, default='long_term_forecast',
                    help='task name, options:[long_term_forecast, short_term_forecast, imputation, classification, anomaly_detection]')
parser.add_argument('--is_training', type=int, default=1, help='status')
parser.add_argument('--model_id', type=str, default='ETTh1_ETTh2_512_192', help='model id')
parser.add_argument('--model_comment', type=str, default='TimeLLM-ETTh1', help='prefix when saving test results')
parser.add_argument('--model', type=str, default='TimeLLM',
                    help='model name, options: [Autoformer, DLinear]')
parser.add_argument('--seed', type=int, default=2021, help='random seed')

# data loader
parser.add_argument('--data', type=str, default='ETTh2', help='dataset type')
parser.add_argument('--data_pretrain', type=str, default='ETTh1', help='dataset type')
parser.add_argument('--root_path', type=str, default='/content/gdrive/MyDrive/TimeLLM/dataset/ETT-small/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh2.csv', help='data file')
parser.add_argument('--data_path_pretrain', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; '
                         'M:multivariate predict multivariate, S: univariate predict univariate, '
                         'MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--loader', type=str, default='modal', help='dataset type')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, '
                         'options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], '
                         'you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='/content/gdrive/MyDrive/TimeLLM/checkpoints', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=512, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=192, help='prediction sequence length')
parser.add_argument('--seasonal_patterns', type=str, default='Monthly', help='subset for M4')

# model define
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size')
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=32, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=128, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=3, help='attn factor')
parser.add_argument('--dropout', type=float, default=0.1, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in encoder')
parser.add_argument('--patch_len', type=int, default=16, help='patch length')
parser.add_argument('--stride', type=int, default=8, help='stride')
parser.add_argument('--prompt_domain', type=int, default=0, help='')
parser.add_argument('--llm_model', type=str, default='LLAMA', help='LLM model') # LLAMA, GPT2, BERT
parser.add_argument('--llm_dim', type=int, default='4096', help='LLM model dimension')# LLama7b:4096; GPT2-small:768; BERT-base:768


# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=1, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=5, help='train epochs')
parser.add_argument('--align_epochs', type=int, default=10, help='alignment epochs')
parser.add_argument('--batch_size', type=int, default=8, help='batch size of train input data')
parser.add_argument('--eval_batch_size', type=int, default=8, help='batch size of model evaluation')
parser.add_argument('--patience', type=int, default=10, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.02, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='Exp', help='exp description')
parser.add_argument('--loss', type=str, default='MSE', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--pct_start', type=float, default=0.2, help='pct_start')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)
parser.add_argument('--llm_layers', type=int, default=32)
parser.add_argument('--percent', type=int, default=100)
parser.add_argument('--content', type=str, default='Description')




_StoreAction(option_strings=['--content'], dest='content', nargs=None, const=None, default='Description', type=<class 'str'>, choices=None, required=False, help=None, metavar=None)

In [9]:

args = parser.parse_args()
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)

# Load the JSON file into a dictionary
with open(os.path.join(timeLLM_path, 'ds_config_zero2.json'), 'r') as f:
    hf_ds_config_dict = json.load(f)


In [10]:

deepspeed_plugin = DeepSpeedPlugin(hf_ds_config= hf_ds_config_dict)
accelerator = Accelerator(kwargs_handlers=[ddp_kwargs], deepspeed_plugin=deepspeed_plugin)


In [ ]:
# Run it
train(args)

TimeLLM init
TimeLLM- LLAMA


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

TimeLLM-model from pretrained
TimeLLM-tokenizer from pretrained
TimeLLM- Tokenizer
TimeLLM-description:
***************
The Electricity Transformer Temperature (ETT) is a crucial indicator in the electric power long-term deployment.
***************

Reprogramming Layer
TimeLLM-output_projection:
***************
FlattenHead(
  (flatten): Flatten(start_dim=-2, end_dim=-1)
  (linear): Linear(in_features=8192, out_features=192, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
***************

***** Content *****
The Electricity Transformer Temperature (ETT) is a crucial indicator in the electric power long-term deployment. This dataset consists of 2 years data from two separated counties in China. To explore the granularity on the Long sequence time-series forecasting (LSTF) problem, different subsets are created, {ETTh1, ETTh2} for 1-hour-level and ETTm1 for 15-minutes-level. Each data point consists of the target value ”oil temperature” and 6 power load features. The train/val/tes

66it [00:27,  2.50it/s]

In [ ]:

parser = argparse.ArgumentParser(description='Time-LLM')

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

# basic config
parser.add_argument('-f', type=str, default='')
parser.add_argument('--task_name', type=str, default='long_term_forecast',
                    help='task name, options:[long_term_forecast, short_term_forecast, imputation, classification, anomaly_detection]')
parser.add_argument('--is_training', type=int, default=1, help='status')
parser.add_argument('--model_id', type=str, default='ETTh1_512_96', help='model id')
parser.add_argument('--model_comment', type=str, default='TimeLLM-ETTh1', help='prefix when saving test results')
parser.add_argument('--model', type=str, default='TimeLLM',
                    help='model name, options: [Autoformer, DLinear]')
parser.add_argument('--seed', type=int, default=2021, help='random seed')

# data loader
parser.add_argument('--data', type=str, default='ETTh1', help='dataset type')
parser.add_argument('--root_path', type=str, default='/content/gdrive/MyDrive/TimeLLM/dataset/ETT-small/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; '
                         'M:multivariate predict multivariate, S: univariate predict univariate, '
                         'MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--loader', type=str, default='modal', help='dataset type')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, '
                         'options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], '
                         'you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='/content/gdrive/MyDrive/TimeLLM/checkpoints', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=512, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')
parser.add_argument('--seasonal_patterns', type=str, default='Monthly', help='subset for M4')

# model define
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size')
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=32, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=128, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=3, help='attn factor')
parser.add_argument('--dropout', type=float, default=0.1, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in encoder')
parser.add_argument('--patch_len', type=int, default=16, help='patch length')
parser.add_argument('--stride', type=int, default=8, help='stride')
parser.add_argument('--prompt_domain', type=int, default=0, help='')
parser.add_argument('--llm_model', type=str, default='LLAMA', help='LLM model') # LLAMA, GPT2, BERT
parser.add_argument('--llm_dim', type=int, default='4096', help='LLM model dimension')# LLama7b:4096; GPT2-small:768; BERT-base:768


# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=1, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=5, help='train epochs')
parser.add_argument('--align_epochs', type=int, default=10, help='alignment epochs')
parser.add_argument('--batch_size', type=int, default=8, help='batch size of train input data')
parser.add_argument('--eval_batch_size', type=int, default=8, help='batch size of model evaluation')
parser.add_argument('--patience', type=int, default=10, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.01, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='Exp', help='exp description')
parser.add_argument('--loss', type=str, default='MSE', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--pct_start', type=float, default=0.2, help='pct_start')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)
parser.add_argument('--llm_layers', type=int, default=32)
parser.add_argument('--percent', type=int, default=100)
parser.add_argument('--content', type=str, default='The Electricity Transformer Temperature (ETT) is a crucial indicator in the electric power long-term deployment.')


args = parser.parse_args()
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)

# Load the JSON file into a dictionary
with open(os.path.join(timeLLM_path, 'ds_config_zero2.json'), 'r') as f:
    hf_ds_config_dict = json.load(f)

#print(hf_ds_config_dict)
#deepspeed_plugin = DeepSpeedPlugin(hf_ds_config_dict)
#accelerator = Accelerator(kwargs_handlers=[ddp_kwargs], deepspeed_plugin=deepspeed_plugin)

#accelerator.prepare()


In [ ]:
accelerator = Accelerator(kwargs_handlers=[ddp_kwargs])


In [ ]:

# Run it
train(args)